# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame


In [54]:
filepath = '../Submission/WeatherData.csv'
df = pd.read_csv(filepath)
df=df.dropna()
df


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Pressure
0,Sioux Lookout,50.1001,-91.9170,-16.53,76.0,0.0,2.30,CA,1.644030e+09,1026.0
1,Sterling,39.0062,-77.4286,33.71,78.0,100.0,1.99,US,1.644029e+09,1020.0
2,Ushuaia,-54.8000,-68.3000,47.86,71.0,0.0,6.91,AR,1.644029e+09,1015.0
3,Arraial do Cabo,-22.9661,-42.0278,75.81,87.0,100.0,22.39,BR,1.644030e+09,1011.0
4,Alta Floresta,-9.8756,-56.0861,71.47,96.0,51.0,2.53,BR,1.644030e+09,1012.0
...,...,...,...,...,...,...,...,...,...,...
650,Brekstad,63.6870,9.6654,35.20,80.0,40.0,39.12,NO,1.644031e+09,976.0
651,Gizo Government Station,-8.1030,156.8419,86.11,71.0,100.0,13.56,SB,1.644031e+09,1005.0
652,Kovdor,67.5662,30.4758,14.94,89.0,100.0,16.80,RU,1.644031e+09,993.0
653,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,72.09,97.0,100.0,5.30,AR,1.644031e+09,1008.0


In [57]:
df=df.dropna()

### Humidity Heatmap


In [61]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#assigning locations to lat, lng
locations = df.loc[:, ["Lat", "Lng"]]

#make map
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df['Humidity'], max_intensity=20, point_radius=5)


fig.add_layer(heatmap_layer)


fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria


In [62]:
df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Pressure
0,Sioux Lookout,50.1001,-91.9170,-16.53,76.0,0.0,2.30,CA,1.644030e+09,1026.0
1,Sterling,39.0062,-77.4286,33.71,78.0,100.0,1.99,US,1.644029e+09,1020.0
2,Ushuaia,-54.8000,-68.3000,47.86,71.0,0.0,6.91,AR,1.644029e+09,1015.0
3,Arraial do Cabo,-22.9661,-42.0278,75.81,87.0,100.0,22.39,BR,1.644030e+09,1011.0
4,Alta Floresta,-9.8756,-56.0861,71.47,96.0,51.0,2.53,BR,1.644030e+09,1012.0


In [32]:
#look for ideal conditions: Max temp between 60 and 70 degrees with wind speed less than 15mph and more than 10% cloudiness
    
mask = (df.Max_Temp <= 70) & (df.Max_Temp >= 60 )  & (df.Wind_Speed < 15) & (df.Cloudiness < 10) & (df.Humidity <30)
df1 = df.loc[mask].reset_index(drop=True)
df1
    
    

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Pressure
0,Niamey,13.5137,2.1098,68.29,13.0,0.0,9.22,NE,1.644030e+09,1015.0
1,Adrar,20.5022,-10.0711,62.76,12.0,0.0,9.73,MR,1.644030e+09,1016.0
2,Mongo,12.1844,18.6930,62.46,18.0,8.0,8.46,TD,1.644030e+09,1014.0
3,Sokoto,13.0609,5.2390,61.50,15.0,0.0,14.99,NG,1.644030e+09,1015.0
4,Yuli,9.6989,10.2744,64.31,22.0,0.0,7.40,NG,1.644030e+09,1014.0
5,Bara,10.3744,10.7288,61.23,23.0,0.0,8.05,NG,1.644030e+09,1015.0
6,Tombouctou,20.0000,-3.0000,60.30,11.0,0.0,9.46,ML,1.644030e+09,1018.0
7,Diapaga,12.0708,1.7889,65.52,15.0,3.0,8.41,BF,1.644030e+09,1013.0
8,Bamako,12.6500,-8.0000,68.41,22.0,0.0,5.75,ML,1.644030e+09,1014.0


### Hotel Map


In [63]:
#api Hotel call on ideal cities

target_radius = 5000
target_type = "lodging"
target_search = 'hotel'
hotel = []

for index, row in df1.iterrows():
    # set up a parameters dictionary
    lat = row['Lat']
    long = row['Lng']
    location = f'{lat}, {long}'
    param = {
        'location': location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key,
    }


  
    show_city = row['City']
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Looking for Hotels in {show_city}: Index {index}.")
    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=param)

        places_data = response.json()

        HotName = places_data['results'][0]['name']
        print(f"Retrieving Results")
        hotel.append(HotName)
    except IndexError:
        print('No Results Found')
        hotel.append('NoResultsFound')

Looking for Hotels in Niamey: Index 0.
Retrieving Results
Looking for Hotels in Adrar: Index 1.
No Results Found
Looking for Hotels in Mongo: Index 2.
Retrieving Results
Looking for Hotels in Sokoto: Index 3.
Retrieving Results
Looking for Hotels in Yuli: Index 4.
No Results Found
Looking for Hotels in Bara: Index 5.
No Results Found
Looking for Hotels in Tombouctou: Index 6.
No Results Found
Looking for Hotels in Diapaga: Index 7.
No Results Found
Looking for Hotels in Bamako: Index 8.
Retrieving Results


In [15]:
#Add hotel list to clean df
hotel_df = pd.DataFrame(hotel, columns = ['Hotel_Name'])
hotel_df

df1['Hotel_Name'] = hotel
hotel_df=df1.copy()

In [16]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Pressure,Hotel_Name
0,Niamey,13.5137,2.1098,68.29,13.0,0.0,9.22,NE,1.644030e+09,1015.0,Homeland Hotel
1,Adrar,20.5022,-10.0711,62.76,12.0,0.0,9.73,MR,1.644030e+09,1016.0,NoResultsFound
2,Mongo,12.1844,18.6930,62.46,18.0,8.0,8.46,TD,1.644030e+09,1014.0,Hotel le Consulat/Camping Touristique
3,Sokoto,13.0609,5.2390,61.50,15.0,0.0,14.99,NG,1.644030e+09,1015.0,Grand Ibro Hotel
4,Yuli,9.6989,10.2744,64.31,22.0,0.0,7.40,NG,1.644030e+09,1014.0,NoResultsFound
5,Bara,10.3744,10.7288,61.23,23.0,0.0,8.05,NG,1.644030e+09,1015.0,NoResultsFound
6,Tombouctou,20.0000,-3.0000,60.30,11.0,0.0,9.46,ML,1.644030e+09,1018.0,NoResultsFound
7,Diapaga,12.0708,1.7889,65.52,15.0,3.0,8.41,BF,1.644030e+09,1013.0,NoResultsFound
8,Bamako,12.6500,-8.0000,68.41,22.0,0.0,5.75,ML,1.644030e+09,1014.0,"Radisson Blu Hotel, Bamako"


In [34]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
Hotel_locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker
markers = gmaps.marker_layer(Hotel_locations, info_box_content=hotel_info)

# heatmap

heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df['Humidity'])

fig = gmaps.figure()
fig.add_layer(markers)
fig.add_layer(heatmap_layer)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))